In [84]:
import cv2
import imutils
from timeit import default_timer as timer

In [88]:
def test_classifier(img_path, img_path_2, classifier_model_path):
    haar_cascade = cv2.CascadeClassifier(classifier_model_path)
    assert(not haar_cascade.empty())
    img = cv2.imread(img_path)
    img_smaller = imutils.resize(img, width = 300)
    start = timer()
    faces_rect = haar_cascade.detectMultiScale(img_smaller, 1.1, 9)
    end = timer()
    diff = end - start 
    print(diff)
    for (x, y, w, h) in faces_rect: 
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
    cv2.imwrite(img_path_2, img)

In [91]:
test_classifier('d:/dane/face_recognition/faces.jpg', 'd:/dane/face_recognition/faces2.jpg', 'd:/Dane/face_recognition/haarcascade_frontalface_default.xml')

0.013974100002087653


In [92]:
test_classifier('d:/dane/face_recognition/faces.jpg', 'd:/dane/face_recognition/faces3.jpg', 'd:/Dane/face_recognition/haarcascade_frontalface_alt.xml')

0.01322930003516376


In [93]:
def test_dnn_detector(model_file, config_file, use_gpu, src_img_path, target_img_path, threshold):
    img = cv2.imread(src_img_path)
    model = cv2.dnn.readNetFromCaffe(config_file, model_file)
    if use_gpu:
        model.setPreferableBackend(cv2.dnn.DNN_TARGET_CPU)
    else:
        model.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
        model.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)  
        
    img_height = img.shape[0]
    img_width = img.shape[1]        
    
    start = timer()
    
    blob = cv2.dnn.blobFromImage(img, 1.0, (300, 300), [104, 117, 123], False, False)
    model.setInput(blob)
    detections = model.forward()
    
    end = timer()
    diff = end - start 
    print(diff)
    
    img_cpy = img.copy()

    boxes = []
    print(detections.shape[2])
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        print(confidence)
        if confidence > threshold:
            x1 = int(detections[0, 0, i, 3] * img_width)
            y1 = int(detections[0, 0, i, 4] * img_height)
            x2 = int(detections[0, 0, i, 5] * img_width)
            y2 = int(detections[0, 0, i, 6] * img_height)
            boxes.append([x1, y1, x2, y2])
            cv2.rectangle(
                img_cpy,
                (x1, y1),
                (x2, y2),
                (0, 255, 0),
                int(round(img_height / 150)),
                8,
            )
        
    cv2.imwrite(target_img_path, img_cpy)    

In [94]:
model_file = "d:/Dane/face_recognition/res10_300x300_ssd_iter_140000.caffemodel"
config_file = "d:/Dane/face_recognition\deploy.prototxt"
src_img_path = "d:/dane/face_recognition/faces.jpg"
target_img_path = 'd:/dane/face_recognition/faces4.jpg'

In [95]:
test_dnn_detector(model_file, config_file, True, src_img_path, target_img_path, 0.3)

0.0409783999202773
200
0.999969
0.9994849
0.9994419
0.9986456
0.9910812
0.63445145
0.39947802
0.12451282
0.11797974
0.11629464
0.11569969
0.11561984
0.113841064
0.10715681
0.10704487
0.10643935
0.10587718
0.105599195
0.10470722
0.10447264
0.10337034
0.10267284
0.1025433
0.102258034
0.1008621
0.099431865
0.09901101
0.097795464
0.09736449
0.09693721
0.09685301
0.09465207
0.09457843
0.094338596
0.094247706
0.09339132
0.09336871
0.09329679
0.09326252
0.09322209
0.092690386
0.09241397
0.09138571
0.090792
0.09076597
0.09076486
0.09072632
0.09054916
0.09039981
0.09021428
0.09018702
0.09015928
0.090021394
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0